In [ ]:
import Pkg;
Pkg.activate(joinpath(@__DIR__, ".."));
Pkg.instantiate();
using DelimitedFiles
using CSV
using LinearAlgebra
using ForwardDiff
using RobotDynamics
using Ipopt
using MathOptInterface
const MOI = MathOptInterface
using Random;

In [ ]:
include("quadratic_cost.jl")
include("planar_quadruped.jl")
include("nlp.jl")
include("moi.jl")
include("costs.jl")
include("constraints.jl")
include("ref_traj.jl");

In [ ]:
h = 2
v_init_y = sqrt(2 * 9.81 * h)

In [ ]:
# Dynamics model
model = PlanarQuadruped()
g, mb, lb, l1, l2 = model.g, model.mb, model.lb, model.l1, model.l2

# Some parameters
dt = 0.009
N = 41
times = range(0, dt * (N - 1), length=N)
n = 15
m = 5

# initial state
xinit = zeros(n)
xinit[1] = -lb / 2            # xb
xinit[2] = sqrt(l1^2 + l2^2)  # yb
xinit[3] = -pi/6              # θ
xinit[6] = -lb                # x2
xinit[9] = -v_init_y          # yb_dot
xinit[10] = -pi/2             # theta_dot
# xinit[12] = -v_init_y         # y1_dot
# xinit[14] = -v_init_y         # y2_dot

# Desired final state
xterm = zeros(n)
xterm[1] = -lb / 2           # xb
xterm[2] = sqrt(l1^2 + l2^2) # yb
xterm[6] = -lb;              # x2

In [ ]:
# Reference Trajectory
Xref, Uref = reference_trajectory(model, N, xterm, dt);

In [ ]:
# Objective
Q = Diagonal([10.0; 10.0; 1.0; 10.0; 10.0; 10.0; 10.0; 10.0; 10.0; 1.0; 10.0; 10.0; 10.0; 10.0; 0.0])
R = Diagonal(fill(1e-3, 5))
R[end, end] = 0.0
Qf = Q

obj = map(1:N-1) do k
    LQRCost(Q, R, Xref[k], Uref[k])
end
push!(obj, LQRCost(Qf, R * 0, Xref[N], Uref[1]));

In [ ]:
# Define the NLP
nlp = HybridNLP(model, obj, N, xinit, xterm);

In [ ]:
# initialize guess
Xguess = [zeros(n) for x in Xref]

for k = 1:N
    Xguess[k][1:14] = xterm[1:14]
    Xguess[k][end] = dt * (k - 1)
end

In [ ]:
Z0 = packZ(nlp, Xguess, Uref)

Z_sol, solver = solve(Z0, nlp, c_tol=1e-3, tol=1e-3)

In [ ]:
Z_sol[1:15] - xinit

In [ ]:
Z_sol[end-14:end] - xterm

In [ ]:
@show F1y_final = Z_sol[797]
@show F2y_final = Z_sol[799]
@show F1y_final + F2y_final

In [ ]:
dt = zeros(N - 1)
for k = 1:N-1
    @show dt[k] = Z_sol[20+20*(k-1)];
end

In [ ]:
F1y = zeros(N-1)
F2y = zeros(N-1)
for k = 1:N-1
    @show F1y[k] = Z_sol[17+20*(k-1)];
    @show F2y[k] = Z_sol[19+20*(k-1)];
end

In [ ]:
ui = nlp.uinds
Z_sol[ui[end]][2] + Z_sol[ui[end]][4] + nlp.model.mb * nlp.model.g

In [ ]:
# display(Z_sol)
writedlm("data_11.csv", Z_sol, ',')